In [1]:
import tensorflow as tf
import pandas as pd
import requests
import numpy as np
import json
from io import BytesIO
from PIL import Image

In [2]:
APP_URL = "http://127.0.0.1:5000/inference"
DATA_PATH = "../data/data.csv"

In [3]:
def get_prediction(img: np.array):
    resp = requests.post(
        APP_URL, 
        json={"image": img.tolist()},
    )
    return np.array(
        resp.json()["prediction"]
    )

In [4]:
# example using first 10 observations from the provided data
# for evaluation the client will be calling the app w/ a test set
urls = pd.read_csv(DATA_PATH)
urls = urls.iloc[:10, :]

pairs = []
for ix, row in urls.iterrows():
    res = []
    for col in ["image_url", "mask_url"]:
        resp = requests.get(row[col])
        img = Image.open(BytesIO(resp.content))
        res.append(np.array(img, dtype=np.uint8))
    pairs.append(res)

In [5]:
precision = tf.keras.metrics.Precision()
recall = tf.keras.metrics.Recall()
iou = tf.keras.metrics.MeanIoU(num_classes=2)

In [6]:
preds = []
for img, _ in pairs:
    preds.append(get_prediction(img))

In [7]:
for pair, pred in zip(pairs, preds):
    img, mask = pair
    precision.update_state(mask, pred)
    recall.update_state(mask, pred)
    iou.update_state(mask, pred)

In [8]:
results = f"""
Precision : {precision.result().numpy():.3f}
Recall : {recall.result().numpy():.3f}
IOU : {iou.result().numpy():.3f}
"""
print(results)


Precision : 0.233
Recall : 0.988
IOU : 0.137

